# 多Agnet


In [1]:
from tabnanny import verbose

from langchain_classic.agents import create_openai_tools_agent, AgentExecutor
from langchain_classic.chains.llm import LLMChain
from langchain_tavily import TavilySearch
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.tools import create_retriever_tool, tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from docutils.nodes import document
from langchain_community.document_loaders import TextLoader, WebBaseLoader
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# 尝试这个导入路径
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

def create_first_agent():
    # 创建一个消息历史对象
    message_history = ChatMessageHistory()
    # 创建一个文档加载器
    loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
    docs = loader.load()
    # 创建一个文档分割器,参数是1000和200，代表每个文档的块大小和块重叠大小
    documents = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200
    ).split_documents(docs)
    # 使用本地嵌入模型替代 OpenAI embeddings
    # 创建一个向量数据库,参数是document和embeddings,代表文档和嵌入模型
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector = FAISS.from_documents(documents, embeddings)
    # 创建一个向量数据库检索器
    retriever = vector.as_retriever()
    # 创建一个向量数据库检索器工具
    retriever_tool = create_retriever_tool(
        retriever,
        "langsmith_search",
        "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
    )

    # 使用新的 TavilySearch 类
    search = TavilySearch()

    tools = [search, retriever_tool]

    llm = ChatOpenAI(
        model="gpt-3.5-turbo:free",
        base_url=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0 # 温度是指的模型输出结果的随机性。分数越高，输出结果越随机。
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant."),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ]
    )
    # 创建一个Agnet
    agent = create_openai_tools_agent(llm, tools, prompt)
    # 创建一个AgentExecutor
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    # Runnable是一个可运行对象，可以运行一个函数，也可以运行一个Agent，lambda用于创建一个可运行对象
    agent_with_chat_history = RunnableWithMessageHistory(
        agent_executor,
        lambda session_id: message_history,
        input_messages_key="input",
        history_messages_key='chat_history',
    )
    return agent_with_chat_history

# 创建一个基础计算器工具
@tool
def basic_calculator(query):
    """Basic calculator tool"""
    try:
        result = eval(query)
        return f"The result is {result}"
    except (SyntaxError, NameError) as e:
        return f"Sorry, I couldn't calculate that due to an error: {e}"


# 创建一个方程求解工具
@tool
def equation_solver(query):
    """Equation solver tool"""
    # Basic equation solver (placeholder)
    # Implement specific logic for solving equations
    return "Equation solver: This feature is under development."


def create_second_agent():
    llm = ChatOpenAI(
        model="gpt-4o-mini:free",
        base_url=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0
    )

    tools = [basic_calculator, equation_solver]

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "你很擅长解决与数学相关的问题。请将问题的答案返回给用户"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ]
    )

    agent = create_openai_tools_agent(llm, tools, prompt)

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    message_history = ChatMessageHistory()
    # 给智能体（Agent）添加 “对话历史记忆” 能力
    agent_with_chat_history = RunnableWithMessageHistory(
        agent_executor,
        lambda session_id: message_history,
        input_messages_key="input",
        history_messages_key='chat_history',
    )
    return agent_with_chat_history

def get_agent(user_input):
     template = "You are a helpful assistant. Classify the user input as either 'math' if it's math-related or 'general/technical' otherwise. respond directly with the classification. Question: {question}"
     prompt = PromptTemplate(template=template, input_variables=["question"])

     llm = ChatOpenAI(
        model="gpt-3.5-turbo:free",
        base_url=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0
    )

     llm_chain = LLMChain(prompt=prompt, llm=llm)

     response = llm_chain.invoke({"question": user_input})
     return  response["text"].strip().lower()  # 标准化返回值

def main():
    load_dotenv()
    # create agents
    technical_agent = create_first_agent()
    math_agent = create_second_agent()
    user_input = input("请输入问题：")

    response = get_agent(user_input)
    if "math" in response:
        print("invoking math agent")
        # Run the math agent
        math_agent.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": "test"}},
        )
    else:
        print("invoking technical agent")
        # Run the technical agent
        technical_agent.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": "test"}},
        )

# 运行主函数
if __name__ == "__main__":
    main()

USER_AGENT environment variable not set, consider setting it to identify your requests.


ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error